In [6]:
import serial
import sys
import glob
import time
import serial.tools.list_ports
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display
from IPython.core.display import HTML
from IPython.display import clear_output

ser = None

def serial_ports():
     result = []
     ports = serial.tools.list_ports.comports()
     for port, desc, hwid in sorted(ports):
          comm =  "{}: {}".format(port, desc)
          result.append(comm) 
     return result

def InitSerial(port, bps = 9600, to = 0):
    global ser
    try:
        ser = serial.Serial(port, bps, timeout = to)  # open serial port
        ser.flushInput()
        ser.flushOutput()
        return ser.name
    except Exception as e:
        return 'ERR: ' + str(e)

def CloseSerial():
    return('done')
    try:
        ser.flush()
        ser.close()
        return 'done'
    except Exception as e:
        return 'ERR: ' + str(e)    

def WriteSerial(string):
    try:
        reply = ser.write(string.encode())
        return str(reply)
    except Exception as e:
        return 'ERR: ' + str(e)    

def ReadSerial():
    try:
        reply = ''
        while ser.in_waiting:
            reply = reply + ser.readline().decode()
        return reply
    except Exception as e:
        return 'ERR: ' +  str(e)
    
def SendRead(string):
    if WriteSerial(string):
        time.sleep(0.1)
        return ReadSerial()
    
updatedPortList = []

def search_for_ports():
    updatedPortList = []
    serialPortList = serial_ports()
    for x in serialPortList:
        updatedPortList.append(x[:x.index(':')])
    return updatedPortList
        
def on_value_change(change):
    with output2:
        InitSerial(change['new'],115200)
        WriteSerial('\x03')
        run_some_code()
        
def run_some_code():
    connectCode = '''
import hub,utime
battery.info()
    '''
    motorTest = '''
import hub,utime
hub.port.A.info()
hub.port.B.info()
    '''
    WriteSerial('\x05')
    WriteSerial(connectCode)
    WriteSerial('\x04')
    WriteSerial('\x03')
    
    time.sleep(1) #wait for everything to get over there and read all the replies
    words = ReadSerial()
#     print(words)
    if (">>>" in words):
        WriteSerial('\x05')
        WriteSerial(motorTest)
        WriteSerial('\x04')
        time.sleep(1) #wait for everything to get over there and read all the replies
        motorwords = ReadSerial()
#         print (motorwords)
        if ('{\'type\': None}' not in motorwords):
            print("You are now connected to SPIKE Prime!")
        else:
            print("Error, are you sure the motors are plugged in to ports A and B?")    
    else:
        print("Error, try reconnecting or connecting to a different serial port.")

def forward_SpeedA (b):
    with goFasterA:
        goForward = '''hub.port.A.motor.pwm(-50)
'''
        WriteSerial('\x05')
        WriteSerial(goForward)
        WriteSerial('\x04')
def backward_SpeedA (b):
    with goSlowerA:
        goBackward = '''hub.port.A.motor.pwm(50)
'''
        WriteSerial('\x05')
        WriteSerial(goBackward)
        WriteSerial('\x04')
def break_Speed(b):
    with stop:
        stopA = '''hub.port.A.motor.brake()
'''
        stopB = '''hub.port.B.motor.brake()
'''
        WriteSerial('\x05')
        WriteSerial(stopA)
        WriteSerial('\x04')
        WriteSerial('\x05')
        WriteSerial(stopB)
        WriteSerial('\x04')

def forward_SpeedB (b):
    with goFasterB:
        goForward = '''hub.port.B.motor.pwm(-50)
'''
        WriteSerial('\x05')
        WriteSerial(goForward)
        WriteSerial('\x04')
def backward_SpeedB (b):
    with goSlowerB:
        print("A")
        goBackward = '''hub.port.B.motor.pwm(50)
'''
        WriteSerial('\x05')
        WriteSerial(goBackward)
        WriteSerial('\x04')

setModes = '''
import hub,utime
hub.port.A.motor.mode(1)
hub.port.A.motor.brake()
hub.port.B.motor.mode(1)
hub.port.B.motor.brake()
'''
WriteSerial('\x05')
WriteSerial(setModes)
WriteSerial('\x04')

forwardButtonA = widgets.Button(
    disabled=False,
    layout = Layout(height = '40px',width = '40px',margin = '0px 0px 0px 0px',padding = '0px 0px 0px 0px'),
    icon='arrow-left' # (FontAwesome names without the `fa-` prefix)
)
goFasterA = widgets.Output()
forwardButtonB = widgets.Button(
    disabled=False,
    layout = forwardButtonA.layout,
    icon='arrow-up' # (FontAwesome names without the `fa-` prefix)
)
goFasterB = widgets.Output()
backwardButtonA = widgets.Button(
    disabled=False,
    layout = forwardButtonA.layout,
    icon='arrow-right' # (FontAwesome names without the `fa-` prefix)
)
goSlowerA = widgets.Output()
backwardButtonB = widgets.Button(
    disabled=False,
    layout = forwardButtonA.layout,
    icon='arrow-down' # (FontAwesome names without the `fa-` prefix)
)
goSlowerB = widgets.Output()

breakButton = widgets.Button(
    disabled=False,
    layout = forwardButtonA.layout,
    icon='stop' # (FontAwesome names without the `fa-` prefix)
)
stop = widgets.Output()

buttonX = widgets.Button(
    disabled=False,
    layout = forwardButtonA.layout
)

def drawingMachineButtons():
    forwardButtonA.on_click(forward_SpeedA)
    forwardButtonB.on_click(forward_SpeedB)
    breakButton.on_click(break_Speed)
    backwardButtonA.on_click(backward_SpeedA)
    backwardButtonB.on_click(backward_SpeedB)
    display(widgets.VBox((widgets.HBox((buttonX,forwardButtonB,buttonX)),widgets.HBox((forwardButtonA,breakButton,backwardButtonA)),widgets.HBox((buttonX,backwardButtonB,buttonX)))))